# Prototype (프로토 타입)

    "기존 객체로부터 객체를 만들 수는 없을까?"

## 정의

클래스의 생성자를 호출하지 않고 기존 객체의 메서드를 통해 기존 객체의 사본 객체를 생성하는 패턴.

클래스를 몰라도 객체를 생성할 수 있다는 점, 기존의 복잡하게 생성된 객체를 쉽게 카피할 수 있다는 점이 유용할 수 있다. 그러나 순환 참조가 있는 객체를 복사하는 건 까다로울 수 있다.

해당 프로토콜은 파이썬의 copy 모듈이 지원한다. 특수 메서드 \_\_copy__, \_\_deepcopy__를 작성하여 얕은 복사와 깊은 복사를 직접 구현할 수 있다.

## 구현

In [28]:
from __future__ import annotations
from copy import copy, deepcopy


class SelfReferencingEntity:
    def __init__(self, parent: SelfReferencingEntity = None) -> None:
        self.parent = parent


class Componenet:

    def __init__(self, immutable, mutable, circular_ref):
        self.immutable = immutable
        self.mutable = mutable
        self.circular_ref = circular_ref

    def __copy__(self):
        # b = copy(a)
        # assert a is not b
        # assert a.x is a.x
        new = type(self)(
            self.immutable,
            self.mutable,
            self.circular_ref
        )
        new.__dict__.update(self.__dict__)
        return new
    
    def __deepcopy__(self, memo: dict = {}):
        # b = deepcopy(a)
        # assert a is not b
        # assert a.x is not a.x
        # assert a.x.y... is not a.x.y...
        new = type(self)(
            self.immutable,
            deepcopy(self.mutable, memo), 
            deepcopy(self.circular_ref, memo)
        )
        new.__dict__.update(deepcopy(self.__dict__, memo))
        return new


if __name__ == "__main__":

    immutable = "abc"
    mutable = [1, {1, 2, 3}, [1, 2, 3]]
    circular_ref = SelfReferencingEntity(parent=SelfReferencingEntity())

    component = Componenet(immutable, mutable, circular_ref)
    shallow_copied_component = copy(component)
    deep_copied_component = deepcopy(component)

    # 모두 서로 다른 id를 지닌 다른 객체들이다.
    assert component is not shallow_copied_component
    assert component is not deep_copied_component

    # 얕은 복사일 경우 내부 속성의 객체는 복사하지는 않는다.
    assert component.mutable is shallow_copied_component.mutable
    assert component.circular_ref is shallow_copied_component.circular_ref
    assert component.circular_ref.parent is shallow_copied_component.circular_ref.parent

    # 깊은 복사일 경우 내부 속성의 객체들도 재귀적으로 새로운 객체로 구성한다.
    assert component.mutable is not deep_copied_component.mutable
    assert component.circular_ref is not deep_copied_component.circular_ref
    assert component.circular_ref.parent is not deep_copied_component.circular_ref.parent